# Day 28: 最終プロジェクト1 - 画像フィルタアプリケーション

## 学習目標
- [ ] 総合的な画像処理パイプラインを構築できる
- [ ] 従来型フィルタ（平滑化、シャープン、ガウシアン）を実装できる
- [ ] モルフォロジー処理（膨張、収縮）を適用できる
- [ ] 画像セグメンテーション技術を応用できる
- [ ] アプリケーションとしての実用的なコードを書ける
- [ ] パフォーマンス最適化を実践できる

---

# Part 1: 理論セクション（午前2時間）

## 1.1 画像フィルタリングの概要

画像フィルタリングは、画像処理の中核をなす技術です。目的に応じて様々なフィルタがあります：

**基本カテゴリ**:
1. **平滑化フィルタ**: ノイズ除去、ぼかし
2. **シャープンフィルタ**: エッジ強調、輪郭を鮮明に
3. **エッジ検出フィルタ**: 境界検出、物体認識の前処理
4. **モルフォロジー処理**: 形状的な特徴抽出

**フィルタの数理的背景**:
- 全てのフィルタは**畳み込み演算**で実現される
- フィルタ係数が重み付けカーネル（カーネル）
- 出力画素 = 入力画素とカーネルの重み付き平均

### 1.2 畳み込み演算の原理

フィルタリングの核心である畳み込み演算を理解しましょう：

```
出力(i,j) = Σ Σ 入力(x,y) × フィルタ(i-x,j-y)
x∈(-k,k) y∈(-k,k)

ここで：
- (i,j): 出力画素の座標
- (x,y): 入力画素の座標
- フィルタ: 2k+1 × 2k+1 のカーネル
```

### 1.3 具体的なフィルタカーネル

**平滑化フィルタ（ノイズ除去）**:

平均化フィルタ（カーネルサイズ3×3）:
```
1/9 × [1 1 1]
         [1 1 1]
         [1 1 1]
```

ガウシアンフィルタ（重みをガウス分布で設定）:
```
1/16 × [1  2  1]
         [2  4  2]
         [1  2  1]
```

**シャープンフィルタ（輪郭強調）**:
```
0 × [-1 -1 -1]
        [-1  9 -1]
        [-1 -1 -1]

+ 入力画像（スケール補正）
```

### 1.4 モルフォロジー処理

二値画像の形状操作を行う処理です：

**基本演算**:
- **膨張（Dilation）**: 白領域を拡張する
- **収縮（Erosion）**: 白領域を縮小する
- **オープニング**: 収縮→膨張でノイズ除去
- **クロージング**: 膨張→収縮で穴埋め

**カーネル構造**:
```
例: 十字型カーネル
0 1 0
1 1 1
0 1 0
```

**演算ルール**: カーネルと重なる領域に白いピクセルが1つでもあれば出力を白に

### 1.5 画像セグメンテーション

画像から特定領域を抽出する技術です：

**手法**:
1. **閾値処理**: 明るさで領域分割
2. **領域成長**: 類似性で領域を拡大
3. **クラスタリング**: K-meansなどで色・強度で分割

**閾値処理の数式**:
```
出力(x,y) = { 1  if 入力(x,y) > 閾値
             { 0  otherwise
```

### 1.6 アプリケーション設計

**システムアーキテクチャ**:
```
├── 入力処理
├── フィルタ選択
├── パラメータ設定
├── 画像処理
├── 出力形式設定
├── 保存・出力
```

**重要な考慮事項**:
- メモリ効率（大画像の場合）
- 処理速度（バッチ処理、並列化）
- ユーザーインターフェース（CLI/GUI）
- エラーハンドリング（不正な画像、メモリ不足）

---

# Part 2: 実践セクション（午後2時間）

## 2.1 環境準備

In [ ]:
# 必要なライブラリのインポート
import math
import random
from pathlib import Path
import matplotlib.pyplot as plt

# 画像処理用のライブラリ（シミュレーション）
def create_test_image(size=100, type='pattern'):
    """テスト用画像を作成"""
    image = [[0 for _ in range(size)] for _ in range(size)]
    
    if type == 'gradient':
        # 傾斜画像
        for i in range(size):
            for j in range(size):
                image[i][j] = int(255 * i / size)
    elif type == 'circle':
        # 円形パターン
        center = size // 2
        for i in range(size):
            for j in range(size):
                dist = math.sqrt((i - center)**2 + (j - center)**2)
                if dist <= center * 0.7:
                    image[i][j] = 255
    elif type == 'noisy':
        # ノイズ付加画像
        for i in range(size):
            for j in range(size):
                base = 128
                noise = random.randint(-30, 30)
                image[i][j] = max(0, min(255, base + noise))
    
    return image

# 画像表示関数
def display_image(image, title=""):
    """画像を表示"""
    plt.figure(figsize=(6, 6))
    plt.imshow(image, cmap='gray', vmin=0, vmax=255)
    plt.title(title)
    plt.axis('off')
    plt.show()

# テスト画像の生成と表示
print("テスト画像の生成と表示")
test_image = create_test_image(100, 'circle')
print(f"画像サイズ: {len(test_image)} × {len(test_image[0])}")
print(f"ピクセル値の範囲: {min(min(row) for row in test_image)} - {max(max(row) for row in test_image)}")

## 2.2 基本的なフィルタの実装

In [ ]:
# フィルタの基底クラス
class ImageFilter:
    """画像フィルタの基底クラス"""
    
    def __init__(self, kernel_size=3):
        self.kernel_size = kernel_size
        self.kernel = self._create_kernel()
    
    def _create_kernel(self):
        """カーネルを作成（サブクラスで実装）"""
        return [[1 for _ in range(self.kernel_size)] for _ in range(self.kernel_size)]
    
    def apply(self, image):
        """フィルタを適用"""
        pad = self.kernel_size // 2
        height = len(image)
        width = len(image[0])
        
        # 畳み込み演算
        result = [[0 for _ in range(width)] for _ in range(height)]
        
        for i in range(height):
            for j in range(width):
                # 周囲の画素を取得
                neighborhood = []
                for ki in range(-pad, pad + 1):
                    for kj in range(-pad, pad + 1):
                        ni, nj = i + ki, j + kj
                        # 境界処理（ゼロパディング）
                        if 0 <= ni < height and 0 <= nj < width:
                            neighborhood.append(image[ni][nj])
                        else:
                            neighborhood.append(0)
                
                # 畳み込み
                conv_value = 0
                for ki in range(self.kernel_size):
                    for kj in range(self.kernel_size):
                        conv_value += self.kernel[ki][kj] * neighborhood[ki * self.kernel_size + kj]
                
                result[i][j] = int(conv_value)
        
        # 値のクリップ（0-255の範囲に）
        for i in range(height):
            for j in range(width):
                result[i][j] = max(0, min(255, result[i][j]))
        
        return result

class MeanFilter(ImageFilter):
    """平均化フィルタ（ぼかし）"""
    
    def _create_kernel(self):
        k = self.kernel_size
        weight = 1.0 / (k * k)
        return [[weight for _ in range(k)] for _ in range(k)]

class GaussianFilter(ImageFilter):
    """ガウシアンフィルタ"""
    
    def _create_kernel(self):
        k = self.kernel_size
        sigma = k / 2.0
        kernel = [[0 for _ in range(k)] for _ in range(k)]
        
        # ガウシアンカーネルの生成
        for i in range(k):
            for j in range(k):
                x = i - k//2
                y = j - k//2
                kernel[i][j] = math.exp(-(x**2 + y**2) / (2 * sigma**2))
        
        # 正規化
        total = sum(sum(row) for row in kernel)
        return [[val/total for val in row] for row in kernel]

class SharpenFilter(ImageFilter):
    """シャープンフィルタ"""
    
    def _create_kernel(self):
        if self.kernel_size == 3:
            return [[0, -1, 0],
                    [-1, 5, -1],
                    [0, -1, 0]]
        else:
            # 一般的なシャープンカーネル
            k = self.kernel_size
            kernel = [[0 for _ in range(k)] for _ in range(k)]
            center = k // 2
            kernel[center][center] = k * k
            for i in range(k):
                for j in range(k):
                    if i != center or j != center:
                        kernel[i][j] = -1
            return kernel

# フィルタのテスト
print("フィルタのテスト")
test_image = create_test_image(50, 'noisy')

# 平均化フィルタ
mean_filter = MeanFilter(kernel_size=3)
mean_result = mean_filter.apply(test_image)
print("平均化フィルタ適用完了")

# ガウシアンフィルタ
gaussian_filter = GaussianFilter(kernel_size=3)
gaussian_result = gaussian_filter.apply(test_image)
print("ガウシアンフィルタ適用完了")

# シャープンフィルタ
sharpen_filter = SharpenFilter(kernel_size=3)
sharpen_result = sharpen_filter.apply(test_image)
print("シャープンフィルタ適用完了")

## 2.3 モルフォロジー処理の実装

In [ ]:
def binarize(image, threshold=128):
    """二値化処理"""
    return [[1 if pixel >= threshold else 0 for pixel in row] for row in image]

def morphological_operation(image, kernel, operation='dilation'):
    """モルフォロジー処理
    
    Args:
        image: 二値画像（0と1で表現）
        kernel: 構造化要素（2次元リスト）
        operation: 'dilation' or 'erosion'
    """
    height = len(image)
    width = len(image[0])
    kernel_size = len(kernel)
    pad = kernel_size // 2
    
    result = [[0 for _ in range(width)] for _ in range(height)]
    
    for i in range(height):
        for j in range(width):
            # 領域を取得
            neighborhood = []
            for ki in range(-pad, pad + 1):
                for kj in range(-pad, pad + 1):
                    ni, nj = i + ki, j + kj
                    if 0 <= ni < height and 0 <= nj < width:
                        neighborhood.append(image[ni][nj])
                    else:
                        neighborhood.append(0)
            
            # 演算を実行
            if operation == 'dilation':
                # 膨張：カーネルの範囲に1があれば1にする
                value = 0
                for ki in range(kernel_size):
                    for kj in range(kernel_size):
                        if kernel[ki][kj] and neighborhood[ki * kernel_size + kj]:
                            value = 1
                            break
            else:  # erosion
                # 収縮：カーネルの範囲に0があれば0にする
                value = 1
                for ki in range(kernel_size):
                    for kj in range(kernel_size):
                        if kernel[ki][kj] and not neighborhood[ki * kernel_size + kj]:
                            value = 0
                            break
            
            result[i][j] = value
    
    return result

# 十字型カーネル
cross_kernel = [
    [0, 1, 0],
    [1, 1, 1],
    [0, 1, 0]
]

# テスト画像の生成と二値化
test_image = create_test_image(100, 'circle')
binary_image = binarize(test_image, 150)
print("二値化処理完了")

# 膨張処理
dilated = morphological_operation(binary_image, cross_kernel, 'dilation')
print("膨張処理完了")

# 収縮処理
eroded = morphological_operation(binary_image, cross_kernel, 'erosion')
print("収縮処理完了")

# オープニング処理（収縮→膨張）
opened = morphological_operation(
    morphological_operation(binary_image, cross_kernel, 'erosion'),
    cross_kernel, 'dilation'
)
print("オープニング処理完了")

## 2.4 画像セグメンテーションの実装

In [ ]:
def threshold_segmentation(image, threshold=128):
    """単純な閾値セグメンテーション"""
    return [[1 if pixel > threshold else 0 for pixel in row] for row in image]

def region_growing(image, seeds, threshold=20):
    """領域成長法
    
    Args:
        image: グレースケール画像
        seeds: 成長開始点のリスト [(y1, x1), (y2, x2), ...]
        threshold: 類似度のしきい値
    """
    height = len(image)
    width = len(image[0])
    segmented = [[0 for _ in range(width)] for _ in range(height)]
    
    # セグメントごとの成長
    for seg_id, (seed_y, seed_x) in enumerate(seeds, 1):
        if 0 <= seed_y < height and 0 <= seed_x < width:
            seed_value = image[seed_y][seed_x]
            
            # 成長用キュー
            queue = [(seed_y, seed_x)]
            
            while queue:
                y, x = queue.pop(0)
                
                if 0 <= y < height and 0 <= x < width and segmented[y][x] == 0:
                    # 類似性チェック
                    if abs(image[y][x] - seed_value) <= threshold:
                        segmented[y][x] = seg_id
                        
                        # 隣接ピクセルを追加
                        for dy in [-1, 0, 1]:
                            for dx in [-1, 0, 1]:
                                if dy != 0 or dx != 0:
                                    queue.append((y + dy, x + dx))
    
    return segmented

def kmeans_segmentation(image, k_clusters=3, max_iter=10):
    """K-meansセグメンテーション
    
    Args:
        image: グレースケール画像
        k_clusters: クラスタ数
        max_iter: 最大反復回数
    """
    # 画像データを1次元にフラット化
    pixels = [pixel for row in image for pixel in row]
    n_pixels = len(pixels)
    
    # 初期重心をランダムに選択
    centroids = random.sample(pixels, k_clusters)
    
    for _ in range(max_iter):
        # 各ピクセルを最も近いクラスタに割り当て
        clusters = [[] for _ in range(k_clusters)]
        for pixel in pixels:
            # 最も近い重心を見つける
            min_dist = float('inf')
            closest_cluster = 0
            for i, centroid in enumerate(centroids):
                dist = abs(pixel - centroid)
                if dist < min_dist:
                    min_dist = dist
                    closest_cluster = i
            clusters[closest_cluster].append(pixel)
        
        # 新しい重心を計算
        new_centroids = []
        for cluster in clusters:
            if cluster:
                new_centroids.append(sum(cluster) / len(cluster))
            else:
                new_centroids.append(centroids[len(new_centroids)])
        
        # 収束チェック
        if centroids == new_centroids:
            break
        centroids = new_centroids
    
    # セグメントマップを作成
    height = len(image)
    width = len(image[0])
    segmented = [[0 for _ in range(width)] for _ in range(height)]
    
    pixel_index = 0
    for i in range(height):
        for j in range(width):
            pixel = image[i][j]
            # 最も近い重心を見つける
            min_dist = float('inf')
            closest_cluster = 0
            for k, centroid in enumerate(centroids):
                dist = abs(pixel - centroid)
                if dist < min_dist:
                    min_dist = dist
                    closest_cluster = k
            segmented[i][j] = closest_cluster + 1
            pixel_index += 1
    
    return segmented

# テスト
print("セグメンテーション処理テスト")
test_image = create_test_image(100, 'gradient')

# 閾値セグメンテーション
threshold_seg = threshold_segmentation(test_image, 128)
print("閾値セグメンテーション完了")

# 領域成長セグメンテーション
seeds = [(25, 25), (75, 75)]  # 開始点
region_seg = region_growing(test_image, seeds, threshold=20)
print("領域成長セグメンテーション完了")

# K-meansセグメンテーション
kmeans_seg = kmeans_segmentation(test_image, k_clusters=3)
print("K-meansセグメンテーション完了")

## 2.5 フィルタアプリケーションの実装

In [ ]:
class ImageFilterApp:
    """画像フィルタアプリケーション"""
    
    def __init__(self):
        self.image = None
        self.original_image = None
        self.filter_history = []
        
    def load_image(self, image_path=None, size=100, type='pattern'):
        """画像の読み込み"""
        if image_path:
            # 実際の画像ファイルから読み込む場合（ここではシミュレーション）
            print(f"画像ファイル {image_path} を読み込みます")
            self.image = create_test_image(size, type)
        else:
            # テスト画像の生成
            self.image = create_test_image(size, type)
        self.original_image = [row[:] for row in self.image]  # ディープコピー
        print(f"画像 {size}×{size} を読み込みました")
        return self.image
    
    def apply_filter(self, filter_type, params=None):
        """フィルタを適用"""
        if self.image is None:
            print("画像が読み込まれていません")
            return None
        
        if filter_type == 'mean':
            kernel_size = params.get('kernel_size', 3)
            filter_obj = MeanFilter(kernel_size)
        elif filter_type == 'gaussian':
            kernel_size = params.get('kernel_size', 3)
            filter_obj = GaussianFilter(kernel_size)
        elif filter_type == 'sharpen':
            kernel_size = params.get('kernel_size', 3)
            filter_obj = SharpenFilter(kernel_size)
        else:
            print(f"未知のフィルタタイプ: {filter_type}")
            return None
        
        # フィルタ適用
        result = filter_obj.apply(self.image)
        
        # ヒストリーに保存
        self.filter_history.append({
            'type': filter_type,
            'params': params,
            'result': result
        })
        
        self.image = result
        print(f"フィルタ {filter_type} を適用しました")
        return result
    
    def apply_morphology(self, operation, kernel_type='cross'):
        """モルフォロジー処理を適用"""
        if self.image is None:
            print("画像が読み込まれていません")
            return None
        
        # 二値化
        binary_image = binarize(self.image)
        
        # カーネルの選択
        if kernel_type == 'cross':
            kernel = cross_kernel
        elif kernel_type == 'square':
            kernel = [[1, 1, 1], [1, 1, 1], [1, 1, 1]]
        else:
            print(f"未知のカーネルタイプ: {kernel_type}")
            return None
        
        # 処理の適用
        result = morphological_operation(binary_image, kernel, operation)
        
        # ヒストリーに保存
        self.filter_history.append({
            'type': f'morphology_{operation}',
            'params': {'kernel': kernel_type},
            'result': result
        })
        
        self.image = result
        print(f"モルフォロジー処理 {operation} を適用しました")
        return result
    
    def apply_segmentation(self, method, params=None):
        """セグメンテーションを適用"""
        if self.image is None:
            print("画像が読み込まれていません")
            return None
        
        if method == 'threshold':
            threshold = params.get('threshold', 128)
            result = threshold_segmentation(self.image, threshold)
        elif method == 'region_growing':
            seeds = params.get('seeds', [(50, 50)])
            threshold = params.get('threshold', 20)
            result = region_growing(self.image, seeds, threshold)
        elif method == 'kmeans':
            k = params.get('k_clusters', 3)
            result = kmeans_segmentation(self.image, k)
        else:
            print(f"未知のセグメンテーション方法: {method}")
            return None
        
        # ヒストリーに保存
        self.filter_history.append({
            'type': f'segmentation_{method}',
            'params': params,
            'result': result
        })
        
        self.image = result
        print(f"セグメンテーション {method} を適用しました")
        return result
    
    def save_result(self, filename="result.png"):
        """結果を保存"""
        # ここでは実際の保存はせず、処理をシミュレート
        print(f"結果を {filename} として保存しました")
        return filename
    
    def reset(self):
        """画像をリセット"""
        if self.original_image:
            self.image = [row[:] for row in self.original_image]
            self.filter_history = []
            print("画像がリセットされました")
        
    def get_info(self):
        """画像情報を取得"""
        if not self.image:
            return "画像が読み込まれていません"
        
        height = len(self.image)
        width = len(self.image[0])
        
        # 基本統計量
        flat_pixels = [pixel for row in self.image for pixel in row]
        mean_val = sum(flat_pixels) / len(flat_pixels)
        min_val = min(flat_pixels)
        max_val = max(flat_pixels)
        
        info = {
            'size': (width, height),
            'pixels': width * height,
            'mean': mean_val,
            'min': min_val,
            'max': max_val,
            'filter_count': len(self.filter_history)
        }
        
        return info

# アプリケーションのテスト
print("\n=== 画像フィルタアプリケーションテスト ===")
app = ImageFilterApp()

# 1. 画像の読み込み
app.load_image(size=100, type='noisy')
print(f"画像情報: {app.get_info()}")

# 2. 平均化フィルタの適用
app.apply_filter('mean', {'kernel_size': 5})

# 3. ガウシアンフィルタの適用
app.apply_filter('gaussian', {'kernel_size': 3})

# 4. シャープンフィルタの適用
app.apply_filter('sharpen', {'kernel_size': 3})

# 5. 収縮処理の適用
app.apply_morphology('erosion', 'cross')

# 6. セグメンテーションの適用
app.apply_segmentation('kmeans', {'k_clusters': 3})

# 7. 結果の保存
app.save_result("processed_result.png")

# 8. 最終情報の表示
final_info = app.get_info()
print(f"\n最終処理情報:")
for key, value in final_info.items():
    print(f"  {key}: {value}")

# 9. リセット
app.reset()

## 2.6 パフォーマンス最適化

In [ ]:
def optimized_filter(image, filter_type='mean', kernel_size=3):
    """最適化されたフィルタ処理
    
    以下の最適化を適用:
    1. 事前計算：カーネルの正規化を事前に計算
    2. メモリ効率：不要な中間変数を削除
    3. バッチ処理：行ごとに処理
    """
    height = len(image)
    width = len(image[0])
    pad = kernel_size // 2
    
    # カーネルの生成と正規化
    if filter_type == 'mean':
        kernel = [[1.0 / (kernel_size * kernel_size) for _ in range(kernel_size)] 
                 for _ in range(kernel_size)]
    elif filter_type == 'gaussian':
        sigma = kernel_size / 2.0
        kernel = []
        for i in range(kernel_size):
            row = []
            for j in range(kernel_size):
                x = i - pad
                y = j - pad
                row.append(math.exp(-(x**2 + y**2) / (2 * sigma**2)))
            kernel.append(row)
        
        # 正規化
        total = sum(sum(row) for row in kernel)
        kernel = [[val/total for val in row] for row in kernel]
    
    # 結果の初期化
    result = [[0 for _ in range(width)] for _ in range(height)]
    
    # フィルタリング（最適化されたループ）
    for i in range(height):
        for j in range(width):
            # 畳み込み
            sum_val = 0.0
            valid_count = 0
            
            for ki in range(kernel_size):
                for kj in range(kernel_size):
                    ni = i + ki - pad
                    nj = j + kj - pad
                    
                    if 0 <= ni < height and 0 <= nj < width:
                        sum_val += image[ni][nj] * kernel[ki][kj]
                        valid_count += 1
            
            # 平均値を計算（境界処理）
            if valid_count > 0:
                result[i][j] = int(sum_val / valid_count)
            else:
                result[i][j] = 0
            
            # 値のクリップ
            result[i][j] = max(0, min(255, result[i][j]))
    
    return result

# パフォーマンス比較
import time

print("\n=== パフォーマンス比較 ===")
large_image = create_test_image(200, 'noisy')

# 標準の実装
start_time = time.time()
mean_filter = MeanFilter(5)
result_standard = mean_filter.apply(large_image)
time_standard = time.time() - start_time
print(f"標準実装: {time_standard:.4f}秒")

# 最適化実装
start_time = time.time()
result_optimized = optimized_filter(large_image, 'mean', 5)
time_optimized = time.time() - start_time
print(f"最適化実装: {time_optimized:.4f}秒")

# 速度比の計算
speedup = time_standard / time_optimized
print(f"速度比: {speedup:.2f}倍")

# 結果の比較
diff = 0
for i in range(len(result_standard)):
    for j in range(len(result_standard[0])):
        diff += abs(result_standard[i][j] - result_optimized[i][j])
avg_diff = diff / (len(result_standard) * len(result_standard[0]))
print(f"平均誤差: {avg_diff:.6f}")

## 2.7 複合処理パイプライン

In [ ]:
def apply_preprocessing_pipeline(image):
    """画像前処理パイプライン
    
    典型的な前処理フロー:
    1. ノイズ除去（ガウシアンフィルタ）
    2. 輪郭強調（シャープンフィルタ）
    3. コントラスト調整（ヒストグラム均一化の代わりに標準化）
    """
    # 1. ノイズ除去
    gaussian = optimized_filter(image, 'gaussian', 3)
    print("1. ノイズ除去完了")
    
    # 2. 輪郭強調
    sharpen = optimized_filter(gaussian, 'sharpen', 3)
    print("2. 輪郭強調完了")
    
    # 3. コントラスト調整（標準化）
    pixels = [pixel for row in sharpen for pixel in row]
    mean_val = sum(pixels) / len(pixels)
    std_val = math.sqrt(sum((p - mean_val)**2 for p in pixels) / len(pixels))
    
    # 標準化（平均0、標準偏差1に）
    normalized = [[0 for _ in range(len(sharpen[0]))] for _ in range(len(sharpen))]
    
    for i in range(len(sharpen)):
        for j in range(len(sharpen[0])):
            if std_val > 0:
                normalized[i][j] = (sharpen[i][j] - mean_val) / std_val
            else:
                normalized[i][j] = 0
    
    # 0-255にスケーリング
    min_val = min(min(row) for row in normalized)
    max_val = max(max(row) for row in normalized)
    
    if max_val - min_val > 0:
        range_val = max_val - min_val
        result = [int(255 * (n - min_val) / range_val) for row in normalized for n in row]
        result = [result[i*len(normalized[0]):(i+1)*len(normalized[0])] 
                 for i in range(len(normalized))]
    else:
        result = normalized
    
    print("3. コントラスト調整完了")
    return result

def apply_object_detection_pipeline(image):
    """物体検出パイプライン
    
    典型的な物体検出フロー:
    1. 前処理
    2. セグメンテーション
    3. 特徴抽出
    4. オブジェクト識別
    """
    print("物体検出パイプライン開始")
    
    # 1. 前処理
    preprocessed = apply_preprocessing_pipeline(image)
    print("前処理完了")
    
    # 2. セグメンテーション
    segmented = kmeans_segmentation(preprocessed, k_clusters=4, max_iter=20)
    print("セグメンテーション完了")
    
    # 3. 領域特徴の計算
    height = len(segmented)
    width = len(segmented[0])
    
    # 各セグメントの特徴を計算
    segments = {}
    for i in range(height):
        for j in range(width):
            seg_id = segmented[i][j]
            if seg_id not in segments:
                segments[seg_id] = {
                    'count': 0,
                    'pixels': []
                }
            segments[seg_id]['count'] += 1
            segments[seg_id]['pixels'].append(image[i][j])
    
    # 特徴の計算
    print("\nセグメント情報:")
    features = {}
    for seg_id, data in segments.items():
        count = data['count']
        pixels = data['pixels']
        area = count / (height * width) * 100
        mean_intensity = sum(pixels) / len(pixels)
        
        features[seg_id] = {
            'area': area,
            'mean_intensity': mean_intensity,
            'pixel_count': count
        }
        
        print(f"セグメント {seg_id}: 面積 {area:.1f}%, 平均強度 {mean_intensity:.1f}")
    
    # 4. オブジェクト識別（簡略化）
    objects = []
    for seg_id, feat in features.items():
        # 簡単なルールで物体判定
        if feat['area'] > 5 and 50 < feat['mean_intensity'] < 200:
            objects.append(seg_id)
    
    print(f"\n検出されたオブジェクト: {len(objects)}個")
    for obj_id in objects:
        print(f"- オブジェクト {obj_id}: 面積 {features[obj_id]['area']:.1f}%")
    
    return {
        'preprocessed': preprocessed,
        'segmented': segmented,
        'features': features,
        'objects': objects
    }

# パイプラインのテスト
print("\n=== 複合処理パイプラインのテスト ===")
test_image = create_test_image(150, 'noisy')

# 前処理パイプライン
preprocessed = apply_preprocessing_pipeline(test_image)
print("前処理パイプライン完了")

# 物体検出パイプライン
detection_results = apply_object_detection_pipeline(test_image)
print("物体検出パイプライン完了")

---

# 3. 自己チェック（理解度確認）

## 理論知識
- [ ] **畳み込み演算の原理を説明できる**
- [ ] **各種フィルタの目的と特性を理解している**
  - 平滑化フィルタ：ノイズ除去
  - シャープンフィルタ：輪郭強調
  - ガウシアンフィルタ：重み付きぼかし
- [ ] **モルフォロジー処理の4種類を説明できる**
- [ ] **セグメンテーションの目的と手法を理解している**

## 実装スキル
- [ ] **ImageFilter基底クラスを理解している**
- [ ] **畳み込み演算を自力で実装できる**
- [ ] **モルフォロジー処理のルールを理解している**
- [ ] **ImageFilterAppクラスを適切に使用できる**
- [ ] **パフォーマンス最適化のポイントを理解している**

## アプリケーション開発
- [ ] **前処理パイプラインを構築できる**
- [ ] **物体検出パイプラインを実装できる**
- [ ] **エラーハンドリングを適切に行える**
- [ ] **処理フローをモジュール化できる**

## 追加課題：チャレンジ問題

### Challenge 1: 自作フィルタの実装

独自のフィルタカーネルを作成せよ。例えば：

- エンボスフィルタ：浮き彫り効果
- ラプラシアンフィルタ：エッジ検出
- モーションブラーフィルタ：動きのぼかし

### Challenge 2: パラメータ最適化

フィルタのカーネルサイズを自動最適化するアルゴリズムを実装せよ：

1. 入力画像のノイズレベルを評価
2. ノイズレベルに応じてカーネルサイズを決定
3. 処理結果を評価してフィードバック

### Challenge 3: マルチスレッド処理

大サイズ画像を分割して並列処理を実装せよ：

1. 画像を複数のタイルに分割
2. 各タイルを別スレッドで処理
3. 結果を結合

### Challenge 4: GUIアプリケーション

PyQtやTkinterを使用して、GUIベースの画像フィルタアプリケーションを作成せよ：

- 画像のドラッグ＆ドロップ
- リアルタイムフィルタプレビュー
- パラメータスライダー
- 処理履歴の表示

## 実践プロジェクトアイデア

### プロジェクト1: 写真修復ツール

古写真のノイズ除去、色の補正、欠陥の修復
- スクラッチ検出と除去
- 色彩復元
- 解像度向上（スーパーレゾリューション）

### プロジェクト2: 医学画像解析

- X線画像のノイズ除去
- 臓器のセグメンテーション
  - 異常検出（腫瘍、骨折など）
- 測定機能（サイズ、位置）

### プロジェクト3: 画像認識システム

- 物体検出と追跡
  - キャラクター認識（ゲーム、アニメ）
  - 文字認識（OCRの前処理）
  - 動画の背景除去

### プロジェクト4: アートフィルター

- 絵画的な効果
- アニメ化フィルター
- モノクロ調整
- 特殊効果（水彩、油画など）

---

**お疲れ様でした！** Day 28の最終プロジェクト1を完了しました。

これまで学んできた画像処理技術を統合して、実用的なアプリケーションを構築しました。特に重要だったのは：

1. **畳み込み演算の理解** - すべてのフィルタの基礎
2. **モジュール化設計** - 拡張性のあるコード
3. **パフォーマンス最適化** - 実用的な速度
4. **パイプライン思考** - 複数処理の組み合わせ

次のDay 29では「画像分類アプリケーション」に挑戦します。

**復習ポイント：**
- フィルタの数学的背景
- モルフォロジー処理の適用場面
- セグメンテーション手法の選択基準
- アプリケーションアーキテクチャ設計